In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime



########################################
# Now import your own stuff
########################################

#From in here
import gw_common as GWc
import gw_movmtn as GWmm

sys.path.append(workdir_ + 'myPythonTools/Utils/')
import AveragingUtils as Av
import PlotUtil as Pu
import utils as uti
import validation_data as Val
import shr_const
from shr_const import ShrConst as Cs

sys.path.append(workdir_ + 'myPythonTools/Plotting/')
import LatLonMaps as LL


sys.path.append(workdir_ + 'PyRegridding/Regridder/')
import var_A_x_B as vAB

sys.path.append(workdir_ + 'PyRegridding/Utils/')
import MakePressures as MkP
import GridUtils as GrU



importlib.reload( GWmm )
importlib.reload( GWc )
importlib.reload( LL )
importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
#####

date=[2010,10,15,0]

year=date[0]
month=date[1]
day=date[2]
hour=date[3]

hour0 = ( hour//6 )*6
ihe = hour - hour0

print( year,month,day,hour0 , ihe)

In [ ]:

timetag='2010-09-0*'

In [ ]:
#----------------------------------------------------
#  Get CAM experiment
#-----------------------------------------------------
expA = "c160mm_ne30pg3_FMTndg_cciasi_x04"
A1 = uti.MakeDict4Exp( exp=expA ,
                     user='juliob_run',
                     subd='hist',  #cfg[x_]['grid'] , 
                     hsPat='cam.h1i',
                     ymdPat= timetag ,
                     verbose=True, open_dataset=True )
A3 = uti.MakeDict4Exp( exp=expA ,
                     user='juliob_run',
                     subd='hist',  #cfg[x_]['grid'] , 
                     hsPat='cam.h3i',
                     ymdPat= timetag ,
                     verbose=True, open_dataset=True )


In [ ]:
A1.X

In [ ]:





# Assuming ds is your xarray dataset and 'latitude' is a variable with dimension 'ncol'
latitude_range = (-85, -50)  # Example latitude range

# Identify the indices where latitude falls within the specified range
#latitude_mask = A1.X['lat'].where(
#    (A1.X['lat'] >= latitude_range[0]) & (A1.X['lat'] <= latitude_range[1]) )

latitude_mask = np.where(
    (A1.X['lat'].values >= latitude_range[0]) & (A1.X['lat'].values <= latitude_range[1]) )



#latitude_mask = latitude_mask.compute()

# Get the indices where the condition is True
#latitude_indices = np.where(latitude_mask.values)[0]

print( len(latitude_mask[0]))
# Select data using these indices
#selected_data = ds.isel(ncol=latitude_indices)

latitude_indices=latitude_mask[0]

#print(selected_data)


In [ ]:
#selected_data = ds.isel(ncol=slice(8000, 11001))

A1.X = A1.X.isel(ncol=latitude_indices )


#selected_data = ds.isel(ncol=slice(8000, 11001))

A3.X = A3.X.isel(ncol=latitude_indices )


In [ ]:
plt.plot(A1.X.TAU_MOVMTN[0,65,:])


In [ ]:
print(7.*np.log(A1.X.lev[42].values/1_000.))

In [ ]:

fig,ax=plt.subplots(1,2,squeeze=False,figsize=(24,8))
ulevs=np.linspace(0,0.00200,num=21)
Cpix=40
mf=ax[0,0].tricontourf( A1.X.lon, A1.X.lat, np.average(A1.X.TAU_MOVMTN[0:10,42,:],axis=0) ,levels=ulevs )
#ax[0,1].set_xlim( (50,150) )
ax[0,0].set_ylim( (-90,90) )
cb=fig.colorbar(mf,ax=ax[0,0])

#ax[0,1].tricontour( E.lon, E.lat, E.U[0,Cpix,:] ,levels=ulevs )
#ax[0,2].set_xlim( (50,150) )
#ax[0,2].set_ylim( (0,60) )

In [ ]:
#######################
# get constants to use here


In [ ]:

pifc = A1.X.PINT[0,:,:].values.T
te = A1.X.TEGW[0,:,:].values.T
uu = A1.X.UEGW[0,:,:].values.T
vv = A1.X.VEGW[0,:,:].values.T
xpwp = A3.X.XPWP_SRC_MOVMTN[0,:].values.T


In [ ]:
band_movmtn = GWc.BandType( ngwv=0, dc=5., kwv=2*np.pi/(100_000.) )

In [ ]:
PP=GWc.PType(ifc=pifc)

In [ ]:
PP.ifc.shape

In [ ]:
pifc.shape
pref_edge=1_000.*pifc[0,:]/pifc[0,93]
alpha = GWc.rad_alphas(pref_edge)
plt.plot( alpha, pref_edge)

In [ ]:
######################
from shr_const import ShrConst as Cs
cpair=Cs.CPDAIR
ncol=te.shape[0]

In [ ]:
rhoi, nm, ni =GWc.gw_prof(ncol, PP, cpair, te)

In [ ]:
print( np.shape( nm ))

In [ ]:
zm = A3.X.Z3[0,:,:].values.T


In [ ]:
print(band_movmtn.ngwv)

In [ ]:
####
ncol,pver = np.shape( zm )
ngwv = band_movmtn.ngwv
netdt=np.zeros( (ncol,pver) )
netdt_shcu=np.zeros( (ncol,pver) )
src_level = np.zeros(ncol, dtype=int)
tend_level = np.zeros(ncol, dtype=int)
ubi=np.zeros( (ncol,pver+1) )
ubm=np.zeros( (ncol,pver) )
tau=np.zeros( (ncol, 2*ngwv+1, pver+1) )
c = np.zeros((ncol, 2*ngwv+1) )
xv= np.zeros( ncol )
yv= np.zeros( ncol )
hdepth= np.zeros( ncol )

aack= GWmm.gw_movmtn_src(ncol , band_movmtn , uu, vv, netdt, netdt_shcu, xpwp , zm, src_level, tend_level, tau, ubm, ubi, xv, yv, c, hdepth)

In [ ]:
poop=np.max( tend_level )
print(poop)

In [ ]:
###########################################################
# Complete init of arguments to gw_drag_prof 
# '#'-commented indicates this was init somewhere above
###########################################################

pcnst=1
# Initialize the integer arrays
#src_level = np.zeros(ncol, dtype=int)
#tend_level = np.zeros(ncol, dtype=int)

# Initialize the real arrays (using float64 for real(r8))
dt = 1.0
#te = np.zeros((ncol, pver))
piln = np.log( PP.ifc )  # Log of interface pressures.
#rhoi = np.zeros((ncol, pver+1))
#nm = np.zeros((ncol, pver))
#ni = np.zeros((ncol, pver+1))
#ubm = np.zeros((ncol, pver))
#ubi = np.zeros((ncol, pver+1))
#xv = np.zeros(ncol)
#yv = np.zeros(ncol)
effgw = np.zeros(ncol) + 1.0

#
#c = np.zeros((ncol, 2*ngwv+1))
kvtt = np.zeros((ncol, pver+1))

# Assuming q has some third dimension, let's set it to 5 for now
q = np.zeros((ncol, pver, pcnst))
dse = np.zeros((ncol, pver))
vramp = np.zeros(1)  # Assuming vramp is a pointer to a single-element array

#tau = np.zeros((ncol, 2*ngwv+1, pver+1))
utgw = np.zeros((ncol, pver))
vtgw = np.zeros((ncol, pver))
ttgw = np.zeros((ncol, pver))

# Assuming qtgw has the same third dimension as q
qtgw = np.zeros((ncol, pver, pcnst))
egwdffi = np.zeros((ncol, pver+1))
gwut = np.zeros((ncol, pver, 2*ngwv+1))

dttdf = np.zeros((ncol, pver))
dttke = np.zeros((ncol, pver))

# Optional arguments
ro_adjust = None # np.zeros((ncol, 2*ngwv+1, pver+1))
kwvrdg = None #np.zeros(ncol)
satfac_in = 2.0  # Assuming 2.0 for backward compatibility
lapply_effgw_in = False
lapply_vdiff = False
tau_diag = np.zeros((ncol, pver+1))

# Example function call
goo = GWc.gw_drag_prof(ncol, band_movmtn, PP , src_level, tend_level, dt, 
                te, vramp,   
                piln, rhoi,    nm,   ni,  ubm,  ubi,  xv,    yv,   
                effgw,      c, kvtt, q,   dse,  tau,  utgw,  vtgw, 
                ttgw, qtgw, egwdffi,   gwut, dttdf, dttke, ro_adjust, 
                kwvrdg, satfac_in, lapply_effgw_in, lapply_vdiff, tau_diag ,
                perform_second_half=False, tau_0_ubc=False, do_vertical_diffusion=False )
"""
goo = gw_drag_prof(
            ncol, band, p, src_level, tend_level, dt,
            t, vramp,
            piln, rhoi, nm, ni, ubm, ubi, xv, yv,
            effgw, c, kvtt, q, dse, tau, utgw, vtgw,
            ttgw, qtgw, egwdffi, gwut, dttdf, dttke, ro_adjust,
            kwvrdg, satfac_in, lapply_effgw_in, lapply_vdiff, tau_diag
            )
"""

In [ ]:
tau.shape

In [ ]:
plt.plot( tau[85,0,:] )
plt.plot( A1.X.TAU_MOVMTN[0,:,85])

In [ ]:
plt.scatter( tau[:,0,78] , A1.X.TAU_MOVMTN[0,78,:] )


In [ ]:
plt.plot( tau[:,0,78] )
plt.plot( A1.X.TAU_MOVMTN[0,78,:] )


In [ ]:
plt.plot( tau[:,0,78] - A1.X.TAU_MOVMTN[0,78,:], 'o' )
plt.xlim( 1000, 1200)


In [ ]:
plt.plot( tau[1125,0,:] , '-o')
plt.plot( A1.X.TAU_DIAG_MOVMTN[0,:,1125] )
plt.plot( A1.X.TAU_MOVMTN[0,:,1125] )

In [ ]:
plt.plot( ubm[1125,:] )
plt.plot( A1.X.UBM_MOVMTN[0,:,1125] )
plt.plot( ubm[1125,:] - A1.X.UBM_MOVMTN[0,:,1125] )

In [ ]:
plt.plot( ni[1125,:] )
plt.plot( A1.X.NIEGW[0,:,1125] )


In [ ]:
A1.X